In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from statistics import mode as moda

In [ ]:
df = pd.read_csv("dados_atendimentos.csv")
df_renda_municipios_2010 = pd.read_csv("dados_renda_municipios_2010.csv")

In [ ]:
df.head()

# Questão 1

## Atendimentos online:

In [ ]:
atendimentos_online = df["usuarioOnline"].sum()
atendimentos_online

## Atendimentos offline:

In [ ]:
atendimentos_offline = df["usuarioOnline"].count() - atendimentos_online
atendimentos_offline

# Questão 2

## Quantidade de tipos de avaliação

In [ ]:
df["avaliacao"].unique()

## Ocorrências por tipo

In [ ]:
ocorrencias_positivas = df[df["avaliacao"] == "positiva"]["avaliacao"].count()
ocorrencias_positivas

In [ ]:
ocorrencias_negativas = df[df["avaliacao"] == "negativa"]["avaliacao"].count()
ocorrencias_negativas

In [ ]:
ocorrencias_n_a = df[df["avaliacao"] == "n/ avaliado"]["avaliacao"].count()
ocorrencias_n_a

# Questão 3

## Quantidades de tipos de erros

In [ ]:
df["codigo"].unique()

## Ocorrências por erro

In [ ]:
ocorrencias_550 = df[df["codigo"] == 550]["codigo"].count()
ocorrencias_550

In [ ]:
ocorrencias_500 = df[df["codigo"] == 500]["codigo"].count()
ocorrencias_500

# Questão 4

## Medidas descritivas

In [ ]:
tempo_de_acesso = df["tempoAcesso"]
pd.DataFrame(tempo_de_acesso).describe()

# 5 – Obter as medidas descritivas em relação ao tempo de acesso por atendimentos ocorrem online e offline. (colunas: tempoAcesso e usuarioOnline).

In [ ]:
def tempoAcesso(groupby):
    return df.groupby(groupby).agg(
    minTempoAcesso=pd.NamedAgg("tempoAcesso", "min"),
    maxTempoAcesso=pd.NamedAgg("tempoAcesso", "max"),
    mediaTempoAcesso=pd.NamedAgg("tempoAcesso", "mean"),
    medianaTempoAcesso=pd.NamedAgg("tempoAcesso", "median"),
    modaTempoAcesso=pd.NamedAgg("tempoAcesso", moda),
    varianciaTempoAcesso=pd.NamedAgg("tempoAcesso", "var"),
    desvioPadraoTempoAcesso=pd.NamedAgg("tempoAcesso", "std"),
    quartil25=pd.NamedAgg("tempoAcesso", lambda x: np.quantile(x, 0.25)),
    quartil50=pd.NamedAgg("tempoAcesso", lambda x: np.quantile(x, 0.50)),
    quartil75=pd.NamedAgg("tempoAcesso", lambda x: np.quantile(x, 0.75)),
).reset_index()

In [ ]:
tempoAcesso("usuarioOnline")

# 6 – Obter as medidas descritivas em relação aos tipos de avaliação. (colunas: tempoAcesso e avaliacao).


In [ ]:
tempoAcesso("avaliacao")

# 7 – Obter as medidas descritivas em relação aos tipos de erros. (colunas: tempoAcesso e codigo).


In [ ]:
tempoAcesso("codigo")

# 8 – Montar uma nuvem de palavras para as mensagens de erro. (coluna: erro).

In [ ]:
text = " ".join(message for message in df["erro"])

In [ ]:
wc = WordCloud(
    background_color="White",
    color_func=lambda *args, **kwargs: "black",
    max_words=50,
    width=800,
    height=400,
).generate(text)

In [ ]:
wc.generate(text)

In [ ]:
plt.figure()
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")

# 9 – Montar nuvem de palavras sobre as mensagens de erro para os tipos de avaliação dos clientes. (colunas: erro e avaliacao).

In [ ]:
def criar_nuvem(avaliacao):
    _df = df[df["avaliacao"] == avaliacao]
    text = " ".join(message for message in _df["erro"])
    wc = WordCloud(
        background_color="White",
        color_func=lambda *args, **kwargs: "black",
        max_words=50,
        width=800,
        height=400,
    )
    wc.generate(text)
    plt.figure()
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")


criar_nuvem("positiva")
criar_nuvem("negativa")
criar_nuvem("n/ avaliado")

# 10 – Na tabela do censo de 2010, obter o índice de Gini por UF (state) (criar um dataframe como df_gini_estado).

In [ ]:
def gini(array: np.ndarray or pd.core.series.Series) -> float:
    """
    Calculates the Gini coefficient of a 1D array.

    Args:
        array (array-like): A 1D array of values.

    Returns:
        float: The Gini coefficient of the array.

    """
    # Convert a pandas series to a numpy array
    if type(array) == pd.core.series.Series:
        array = array.to_numpy()

    # Flatten the array if it is not already 1D
    array = array.flatten()

    # Convert the array to float64 if it is not already
    if array.dtype != np.float64:
        array = np.ascontiguousarray(array, dtype=np.float64)

    # Shift the values in the array so that the minimum value is 0
    if np.amin(array) < 0:
        array -= np.amin(array)

    array += 0.0000001
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    return (np.sum((2 * index - n - 1) * array)) / (n * np.sum(array))


df_gini_estado = (
    df_renda_municipios_2010.groupby("state")
    .agg(
        gini=pd.NamedAgg("RDPC", gini),
    )
    .reset_index()
)

df_gini_estado

# 11 – Realizar o join das tabelas de atendimento e df_gini_estado.

In [ ]:
atendimento_x_gini_estado = pd.merge(df, df_gini_estado, on="state", how="inner")
atendimento_x_gini_estado

# 12 – Obter as medidas descritivas em relação ao índice de gini por tipo de avaliação (colunas: indice_gini e avaliacao). Esta análise é importante para conhecer se o banco realiza melhor atendimento em estados de maior diferença social. Pode ser um problema para a imagem do banco.

In [ ]:
atendimento_x_gini_estado.groupby("avaliacao").agg(
    min=pd.NamedAgg("gini", "min"),
    max=pd.NamedAgg("gini", "max"),
    media=pd.NamedAgg("gini", "mean"),
    mediana=pd.NamedAgg("gini", "median"),
    moda=pd.NamedAgg("gini", moda),
    variancia=pd.NamedAgg("gini", "var"),
    desvioPadrao=pd.NamedAgg("gini", "std"),
    quartil25=pd.NamedAgg("gini", lambda x: np.quantile(x, 0.25)),
    quartil50=pd.NamedAgg("gini", lambda x: np.quantile(x, 0.50)),
    quartil75=pd.NamedAgg("gini", lambda x: np.quantile(x, 0.75)),
).reset_index()

## 13 – Escreva um breve resumo das suas análises demonstrando os pontos mais relevantes.

Nesse dataframe foram analisados 1000 usuários.
Onde 74,6% dos casos de atendimentos foram offline e apenas 25,4% tiveram um atendimento online.
É importante avaliar que 51% dos casos de avaliação foram negativas, 25,1% foi positivo e 23,9% decidiram não avaliar. Com esses dados é possivel compreender que o aplicativo não está conseguindo obter resultados favoráveis 
Comentasse também que os dados não podem ser conclusiveis sobre positivos e negativos, pois há muitos usuários que decidiram não avaliar e isso afeta diretamente os resultados. Porém, é possivel dizer que a situação não poderia se tornar agrádavel mesmo que todos que não avaliacem decidicem dar um positivo, pois não ultrapassaria a quantidade de avaliações negativas.
A maior quantidade de erros obtidos foi do 550 sendo então 53,3%.

A mediana de tempo de acesso foi de 246s, sendo o minimo registrado 1s e o máximo de 629s
Detalhadamente:
É possível dizer entender o porque há muito mais casos de atendimentos offline usando o tempo de acesso, a mediana de tempo de acesso offline é de 244s com um minímo de 1s e máximo de 563 enquanto o Online tem uma mediana de 251s um mínimo de 7 e máximo de 629, assim podesse concluir que o atendimento offline é muito mais rápido.
Um dado estranho é que os usuários que avaliaram positivamente demoraram muito mais para concluir suas ações que os que avaliaram negativamente.

ERROS
Com os erros, é possível analisar que os erros que aparecem mais apresentam as mensagens "TENTE NOVAMENTE", "NOVAMENTE MAIS", "MAIS TARDE". Essas mesmas mensagens aparecem para as avaliações relacionadas aos erros.

A respeito das avaliações

Quanto maior a renda municipal maior a chance da avaliação ser negativa, quanto menor maior a chance de ser uma avaliação positiva ou não existir nenhuma avaliação, a moda aponta tal fato.

Isso indica que pessoas com maiores rendas tem mais problemas ao utilizar a plataforma.

